<a href="https://colab.research.google.com/github/tashwinj2004/COHORT_11-IIMSTC/blob/main/cleaned_food_waste.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv('/content/Grocery_Inventory_and_Sales_Dataset (4).csv')

In [5]:
df = df.dropna()

In [6]:
df['Unit_Price'] = df['Unit_Price'].str.replace('$', '', regex=False).str.strip()
df['Unit_Price'] = pd.to_numeric(df['Unit_Price'])

In [7]:
df['Date_Received'] = pd.to_datetime(df['Date_Received'])
df['Expiration_Date'] = pd.to_datetime(df['Expiration_Date'])


In [8]:
df['Shelf_Life_Days'] = (df['Expiration_Date'] - df['Date_Received']).dt.days

In [9]:
df = df[df['Shelf_Life_Days'] > 0]

In [10]:
df = df[df['Stock_Quantity'] >= df['Sales_Volume']]


In [11]:
def calculate_waste(row):

    denominator = row['Inventory_Turnover_Rate'] * row['Shelf_Life_Days']
    if denominator == 0:
        return 0
    return ((row['Stock_Quantity'] - row['Sales_Volume']) / denominator) * 10

df['Food_Waste_kg'] = df.apply(calculate_waste, axis=1)


In [12]:
cols_to_keep = [
    'Catagory',
    'Date_Received',
    'Expiration_Date',
    'Stock_Quantity',
    'Unit_Price',
    'Sales_Volume',
    'Inventory_Turnover_Rate',
    'Shelf_Life_Days',
    'Food_Waste_kg'
]
df_cleaned = df[cols_to_keep].copy()

In [13]:
df_cleaned['Date_Received'] = df_cleaned['Date_Received'].dt.strftime('%Y-%m-%d')
df_cleaned['Expiration_Date'] = df_cleaned['Expiration_Date'].dt.strftime('%Y-%m-%d')


In [14]:
df_cleaned.to_csv('cleaned_food_waste_data_final.csv', index=False)

In [15]:
print(f"Cleaning complete. {len(df_cleaned)} rows remain.")
print(df_cleaned.head())

Cleaning complete. 220 rows remain.
               Catagory Date_Received Expiration_Date  Stock_Quantity  \
6           Oils & Fats    2024-03-18      2024-06-20              96   
7                 Dairy    2025-02-03      2025-02-05              44   
8                 Dairy    2024-12-04      2025-01-08              91   
13  Fruits & Vegetables    2024-04-05      2024-05-06              95   
15                Dairy    2024-02-27      2024-10-05              94   

    Unit_Price  Sales_Volume  Inventory_Turnover_Rate  Shelf_Life_Days  \
6          2.5            67                       13               94   
7          2.5            21                       91                2   
8          3.0            56                       90               35   
13         4.0            69                        5               31   
15         7.0            20                       24              221   

    Food_Waste_kg  
6        0.237316  
7        1.263736  
8        0.111111  
